In [10]:
import openai
import json
import os
import dotenv
dotenv.load_dotenv()

api_key = os.getenv("API_KEY")
base_url=os.getenv("BASE_URL")


In [12]:
client = openai.OpenAI(
    base_url = base_url,
    api_key = api_key
)


In [13]:
messages = [
    {"role": "system", "content": f"You are a helpful assistant with access to functions." 
     															"Use them if required."},
    {"role": "user", "content": "What are Nike's net income in 2022?"}
]


In [14]:
tools = [
    {
        "type": "function",
        "function": {
            # name of the function 
            "name": "get_financial_data",
            # a good, detailed description for what the function is supposed to do
            "description": "Get financial data for a company given the metric and year.",
            # a well defined json schema: https://json-schema.org/learn/getting-started-step-by-step#define
            "parameters": {
                # for OpenAI compatibility, we always declare a top level object for the parameters of the function
                "type": "object",
                # the properties for the object would be any arguments you want to provide to the function
                "properties": {
                    "metric": {
                        # JSON Schema supports string, number, integer, object, array, boolean and null
                        # for more information, please check out https://json-schema.org/understanding-json-schema/reference/type
                        "type": "string",
                        # You can restrict the space of possible values in an JSON Schema
                        # you can check out https://json-schema.org/understanding-json-schema/reference/enum for more examples on how enum works
                        "enum": ["net_income", "revenue", "ebdita"],
                    },
                    "financial_year": {
                        "type": "integer", 
                        # If the model does not understand how it is supposed to fill the field, a good description goes a long way 
                        "description": "Year for which we want to get financial data."
                    },
                    "company": {
                        "type": "string",
                        "description": "Name of the company for which we want to get financial data."
                    }
                },
                # You can specify which of the properties from above are required
                # for more info on `required` field, please check https://json-schema.org/understanding-json-schema/reference/object#required
                "required": ["metric", "financial_year", "company"],
            },
        },
    }
]



In [15]:
chat_completion = client.chat.completions.create(
    model="accounts/fireworks/models/firefunction-v2",
    messages=messages,
    tools=tools,
    temperature=0.1
)


In [16]:
print(chat_completion.choices[0].message.model_dump_json(indent=4))

{
    "content": null,
    "refusal": null,
    "role": "assistant",
    "function_call": null,
    "tool_calls": [
        {
            "id": "call_vBVNNINMpfyfN01TpCAjeWn7",
            "function": {
                "arguments": "{\"metric\": \"net_income\", \"financial_year\": 2022, \"company\": \"Nike\"}",
                "name": "get_financial_data"
            },
            "type": "function",
            "index": 0
        }
    ]
}


In [17]:
####Grammar Mode#####

diagnosis_grammar = """
root      ::= diagnosis
diagnosis ::= "arthritis" | "dengue" | "urinary tract infection" | "impetigo" | "cervical spondylosis"
"""

chat_completion = client.chat.completions.create(
    model="accounts/fireworks/models/llama-v3p1-405b-instruct",
    response_format={"type": "grammar", "grammar": diagnosis_grammar},
    messages=[
        {
            "role": "system",
            "content": "Given the symptoms try to guess the possible diagnosis. Possible choices: arthritis, dengue, urinary tract infection, impetigo, cervical spondylosis. Answer with a single word",
        },
        {
            "role": "user",
            "content": "I have been having trouble with my muscles and joints. My neck is really tight and my muscles feel weak. I have swollen joints and it is hard to move around without becoming stiff. It is also really uncomfortable to walk.",
        },
    ],
)
print(chat_completion.choices[0].message.content)


arthritis


In [18]:

cjk_grammar = """
root        ::= jp-char+ ([ \t\n] jp-char+)*
jp-char     ::= hiragana | katakana | punctuation | cjk
hiragana    ::= [ぁ-ゟ]
katakana    ::= [ァ-ヿ]
punctuation ::= [、-〾]
cjk         ::= [一-鿿]
"""
chat_completion = client.chat.completions.create(
    model="accounts/fireworks/models/llama-v3p1-405b-instruct",
    response_format={"type": "grammar", "grammar": cjk_grammar},
    messages=[
        {
            "role": "user",
            "content": "You are a Japanese tour guide who speaks fluent Japanese. Please tell me what are some good places for me to visit in Kyoto",
        },
    ],
)




print(chat_completion.choices[0].message.content)



おはようございます、ようこそ京都へです。京都は、日本の文化、歴史、自然が集まっている素晴らしい都市です。ここでは、京都のいくつかの人気スポットをご紹介しますね。まず、伏見稲荷大社は、京都で最も有名な神社のひとつです。数千の朱塗りの鳥居が山腹に並び、見る人を圧倒します。次に、清水寺は、京都で最も古い寺院のひとつです。階段状の庭園が美しく、特に紅葉の時期には、見事な景色を楽しむことができます。金閣寺は、金色に輝く寺院で、美しい庭園とともに、京都の代表的な観光スポットです。嵐山は、紅葉の名所として有名で、特に十一月には、美しい景色を楽しむことができます。また、京都の地元食を楽しむことができるグルメスポットもあります。京都には、伝統的な日本料理から、現代的な日本料理まで、様々なグルメが楽しめます。最後に、夜の京都を楽しむために、祇園は、伝統的な日本の文化を感じることができる場所です。伝統的な日本建築や、芸妓さんの踊りなど、京都の夜の雰囲気を味わうことができます。


In [19]:
import openai
from pydantic import BaseModel, Field


class Result(BaseModel):
    winner: str

chat_completion = client.chat.completions.create(
    model="accounts/fireworks/models/mixtral-8x7b-instruct",
    response_format={"type": "json_object", "schema": Result.model_json_schema()},
    messages=[
        {
            "role": "user",
            "content": "Who won the US presidential election in 2012? Reply just in one JSON.",
        },
    ],
)


In [20]:
print(repr(chat_completion.choices[0].message.content))

'{\n"winner": "Barack Obama"\n}'
